# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [11]:
! pip install opencv-python

moviepy 0.2.3.2 has requirement decorator==4.0.11, but you'll have decorator 4.1.0 which is incompatible.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [12]:
import cv2
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [3]:
import os
os.getcwd()

'C:\\Users\\Harsh Modi\\Downloads\\dlnd'

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [4]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



In [5]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 101715
Unique characters: 90
Vectorization...


# Building the network
Our network is a single LSTM layer followed by a Dense classifier and softmax over all possible characters. But let us note that recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in recent times.

In [6]:
import keras

from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

d:\anaconda\envs\stable\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
import random
import sys

for epoch in range(1, 35):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
101715/101715 [==============================] - 66s 654us/step - loss: 1.9524
--- Generating with seed: "es?
Moe_Szyslak: (LOWERING BINOCULARS, SINISTER) Nobody gets"
------ temperature: 0.2
es?
Moe_Szyslak: (LOWERING BINOCULARS, SINISTER) Nobody gets a prow the for the frees a seen the seen a the sell the like my the seed a don't seen my bet a chere a she to the be to the seen a been a fore a chere the cally don't seend the bare the like my my my payen a mone me the bare to be my fould my meat the figet a dould the seent to a the seent it the bare the for the frome the seen my for a the seed the seed the selly the seed the seen the frout a me
------ temperature: 0.5
the seed the seed the selly the seed the seen the frout a me up to some the wally?
Homer_Simpson: (COUGHAS) And shey that sould sive a serome the meat you dound where the seeple mean my ffacty fould a fromed be my way be frieny his the be to seed the filin' secanend fifed the seely for a stight on a lest

Scaur: Notey, Homer muguid.
Apu_Nahas.teeppelly: Lot's not in ad you dos!
Moe_Szyslak: Nor pee!
Homer_Simpson: Anot's altant! Ohresapuriget, repemtteds,, Or My Macutioul urakic.
Carl_oy_na, If Gimbe. Aw, I get take a munch Tob Tokey time to leen in a Manquinlibove." Well, the 
Moe_Szyslak: Selen, a waghtige touse sty! Shaffielvin, 
epoch 5
Epoch 1/1
101715/101715 [==============================] - 64s 633us/step - loss: 1.4543
--- Generating with seed: "(SOTTO; DESPERATE) Here's $42 -- it's everything I have. Run"
------ temperature: 0.2
(SOTTO; DESPERATE) Here's $42 -- it's everything I have. Rundred the love to the bar the secret to have to the book.
Homer_Simpson: (

d:\anaconda\envs\stable\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


SINGS) Oh, I want to have to got to the should better the strainker that some are work the look the bar the sees to the care to want to the care to the stook to have to the care to this the place to the look the bar to the bar to the last to good to do the cat to the should be to the part the shore where the say the stook a l
------ temperature: 0.5
 should be to the part the shore where the say the stook a loser the birst a loh.
Moe_Szyslak: Every, here you get me to make the look your fataning it to the bound anything the mad to make to got the ssort the sate your wait the warl the sirther is the should sure me to good suspreds.
Moe_Szyslak: (SOPS) I got a bodie.
Moe_Szyslak: Hey, I gotta have to have to a doll come this a lotter since make a disting to to have to the Barney hope here? She have his 
------ temperature: 1.0
 a disting to to have to the Barney hope here? She have his the bar toright it.
Moe_Szyslak: (h.hS ROW IT SEES GUGHS YOU MOESLY) Uh, I ouss I've not just ootze Ip 

101715/101715 [==============================] - 63s 616us/step - loss: 1.3369
--- Generating with seed: "ED) And on Christmas.
Barney_Gumble: Yeah. Jesus must be spi"
------ temperature: 0.2
ED) And on Christmas.
Barney_Gumble: Yeah. Jesus must be spill the parters.
Homer_Simpson: (SINGING) That was a lot on the man the part.
Moe_Szyslak: (SIND MOE) I think the leart the leart me the leart man the stood the bart.
Moe_Szyslak: (SINGING) AND SOUSTERIED CAM AND AND THE BARLY THAN MOE) And your ever beer in the matter what your beer the leart on the princess and the bar the look the meaning and the plare. (SINGING) That was a dink the listle the s
------ temperature: 0.5
ng and the plare. (SINGING) That was a dink the listle the stoning pickly and the perfecting a new body the and unte the recille the recall. Drynick the freavan.
Homer_Simpson: (BIRUS) That. I was a good the last beer on the drink coming the Caren.
Moe_Szyslak: Sorry, I'll seen what a looking the leart the late the drink 

...

Moe_Szyslak: Guy you couldal beer plance for moronal ranters of guysen! Mmm, pringfiel storeds, I don't you faved. Ew, thy Homer, 

Moe_Szyslak: We threw it's seys be the Fellsontin.
MANTIRD: Partl-Abarl, sorobasd Drilderfuanters!
Carl_Carlsong: 
------ temperature: 1.2
TIRD: Partl-Abarl, sorobasd Drilderfuanters!
Carl_Carlsong: Gen. What I ieving could get peoneder, into the wince your mant worst bar in my luttle je'rPDen off teernewlingle happinedte's hildhusbup every callt that. (HINDS) I need shana? I could had a mithimes gums, he lookly, Houhk, home you do up "Grondys" -- M-INu_HGEMBrfT----- / CONeTsI Ever see out, HaR... ligoe woulenin' boving that pictuent we have of...yell?" Tar Filleediting! I'm muce his be for t
epoch 17
Epoch 1/1
101715/101715 [==============================] - 61s 602us/step - loss: 1.2236
--- Generating with seed: " to cheat at blackjack.
Homer_Simpson: Boy, you don't need t"
------ temperature: 0.2
 to cheat at blackjack.
Homer_Simpson: Boy, you don't nee

Moe_Szyslak: (SINGS) ANN MOER) I can't believe in the thing never too?
Moe_Szyslak: (SINGS) AME. (SADWERING TOW AND SEARTING AND TALE SCAL
------ temperature: 1.0
slak: (SINGS) AME. (SADWERING TOW AND SEARTING AND TALE SCAL LOW ANY, HSK.Y ANNSOGHTER CHOK-COLLY THE BONG TOU THY YAMU--HALL Yo... (MARECAFLE (WAINING YuBESING ALEs) That's why rope a supixas upurk an uhbar like of Spronking.
Mirggarr_Pring: (CONY SAUP 
Mh.
Moe_Szyslak: Wows it a mod Bors, let's them their guys that nakeg bike with a creecket in the Loss to me to the unds that was brick morow. Ugs?
Homer's_Simpson: I've feat be Eisped!..Ow Coghassiam, Wyl
------ temperature: 1.2
s?
Homer's_Simpson: I've feat be Eisped!..Ow Coghassiam, Wyley, isn't nok. About Mr. Tr?!
Homer_Simpson: (STRAUGHS) The Groblestips of that's andurtadfund then Ame it I cand hes onchs that spitchiliking a minamue.
Moe_Szyslak: Greaty was right.
Lisa_Simpson: Cellald? Cou, ole of theere... We're talkin' changerige did ".


Ma'Stard: Ase. Homery?
Homer

Moe_Szyslak: Oh, you'd looking up to have to the Smile?
Moe_Szyslak: At the Sanutsors. We'll the meling a guy the s Homer. The hand one has working the mi!


Homer_Simpson: I gotta go to like me the loved man...
Homer_Simpson: You got a manchile.
Homer_Simpson: I was say to the loves.
Lenny_Leonard: (SHITTING) And you get to the frishboll strome. Man... that's one of the loved and vollack of hipps. But some are
------ temperature: 1.0
. that's one of the loved and vollack of hipps. But some are in enjome some _Ready of Duffman... is to play he think the end anyweing up ageast!
Barney_Gumble: Well, in the crail Tollames in this ndrick. Wow!
Barney_Gumble: That's beer toh, next coman atering one han's right no Homer annoh hop of a war Margh, dare's out.
The_Rechine: Mayemalive boy mo?
Carl_Carlson: Them we're me the rod!
Bart_Simpson: I CANNYOURT AASED WHAST EVER)
Homer_Simpson: Louss! I 
------ temperature: 1.2
pson: I CANNYOURT AASED WHAST EVER)
Homer_Simpson: Louss! I am drink use. Ho

Moe_Szyslak: (CHUCKLES) What are you a great on a day and the restion.
Moe_Szyslak: (SWEENSS) What thew, I was breal me to the world and I'm gonna show a meance you say to show you a good and I'm gonna show with a beer.
Homer_Simpson: (SNOUENY BEEN) What are you get you mean and the bar to be fo
------ temperature: 0.5
SNOUENY BEEN) What are you get you mean and the bar to be for a meanin"-O-- I don't wanna be for me down to a brakin' a stoo?
Carl_Carlson: (GRUS) I had a man when you doney to the let me to all the nextrend stook me.
Homer_Simpson: (CHANTINGS) Hey, I never din' a but so stay offer its but a coup off and me!
Moe_Szyslak: (TO SELF, LESE) Hey, some heat sure, Manvit Manaim" !
Moe_Szyslak: Wow! Goom Moe, a can as the look for a gank to show we can be friend!

------ temperature: 1.0
Moe, a can as the look for a gank to show we can be friend!
Moe_Szyslak: (SHOCKED) Cought in there, Artion our kis.
Homer_Simpson: Hey, it's dren off Een Socusing!
Moe_Szyslak: Oh, come nothing 

101715/101715 [==============================] - 62s 612us/step - loss: 1.1274
--- Generating with seed: "aying the market. Buy low, sell high, that's my motto. I may"
------ temperature: 0.2
aying the market. Buy low, sell high, that's my motto. I may of the love to the stuff I got a beer and the life in the life.
Moe_Szyslak: (SINGS) MONSTER MOES WING YOU DUYED A TO PAME'Y) (TO SENTS THEN) (SINGS) MONSTER MOES HOMER) I want me to the sign for the scould stay press to me to sell the Springfield Carl can Alder, I don't wanna hate a meaute you have a big beer?
Moe_Szyslak: (SINGS) STIPS) (CALLING TONED NOISE)
Moe_Szyslak: (SINGS) MONSTER A BEERS
------ temperature: 0.5
) (CALLING TONED NOISE)
Moe_Szyslak: (SINGS) MONSTER A BEERS... (SINGS) STEES (ROPERATED NOISE) What do you there. (SINGS) STEES BARMOOK AND THE BONSTER TO YALLY) Shells Simpson?
Moe_Szyslak: Well, the more complain of secome out the best bar too see of the life a million subby is a million to take but the card.
Marge_Sim

Reference : http://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb